# Visualizacion Interactiva de Datos Geoespaciales - Pasto

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/GeoIA-Pasto/notebooks/07_Visualizacion_Interactiva_Pasto.ipynb)

---

## Objetivos

1. Crear mapas interactivos con leafmap
2. Visualizar datos raster y vectoriales
3. Crear mapas divididos (split maps) para comparaciones
4. Exportar mapas en diferentes formatos

## 1. Instalacion de Dependencias

In [ ]:
%pip install -q geoai-py leafmap localtileserver maplibre geopandas folium

## 2. Importacion de Librerias

In [ ]:
import geoai
import leafmap
import leafmap.foliumap as foliumap
import geopandas as gpd
from shapely.geometry import Point, Polygon, box
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

print(f"GeoAI version: {geoai.__version__}")

## 3. Coordenadas de San Juan de Pasto

In [ ]:
# Coordenadas principales
PASTO_CENTER = {"lat": 1.2136, "lon": -77.2811}
PASTO_BBOX = (-77.35, 1.15, -77.20, 1.30)

# Puntos de interes
LUGARES = {
    "Plaza de Narino": {
        "lat": 1.2136,
        "lon": -77.2811,
        "tipo": "Plaza",
        "descripcion": "Plaza principal de Pasto",
    },
    "Universidad de Narino": {
        "lat": 1.2175,
        "lon": -77.2783,
        "tipo": "Educacion",
        "descripcion": "Universidad publica",
    },
    "Volcan Galeras": {
        "lat": 1.2208,
        "lon": -77.3592,
        "tipo": "Volcan",
        "descripcion": "Volcan activo, 4276 m",
    },
    "Laguna de la Cocha": {
        "lat": 1.1167,
        "lon": -77.1500,
        "tipo": "Laguna",
        "descripcion": "Segunda laguna mas grande de Colombia",
    },
    "Aeropuerto Antonio Narino": {
        "lat": 1.3961,
        "lon": -77.2914,
        "tipo": "Aeropuerto",
        "descripcion": "Aeropuerto de Pasto",
    },
    "Estadio Libertad": {
        "lat": 1.2089,
        "lon": -77.2756,
        "tipo": "Deportivo",
        "descripcion": "Estadio de futbol",
    },
    "Terminal de Transportes": {
        "lat": 1.2047,
        "lon": -77.2683,
        "tipo": "Transporte",
        "descripcion": "Terminal terrestre",
    },
    "Centro Comercial Unicentro": {
        "lat": 1.2003,
        "lon": -77.2731,
        "tipo": "Comercial",
        "descripcion": "Centro comercial",
    },
}

# Crear GeoDataFrame
gdf_lugares = gpd.GeoDataFrame(
    {
        "nombre": list(LUGARES.keys()),
        "tipo": [v["tipo"] for v in LUGARES.values()],
        "descripcion": [v["descripcion"] for v in LUGARES.values()],
    },
    geometry=[Point(v["lon"], v["lat"]) for v in LUGARES.values()],
    crs="EPSG:4326",
)

print(f"Puntos de interes: {len(gdf_lugares)}")
gdf_lugares

## 4. Mapa Basico con Diferentes Estilos

In [ ]:
# Mapa con OpenStreetMap
m1 = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=13, height="500px"
)
m1.add_basemap("OpenStreetMap")
m1.add_title("San Juan de Pasto - OpenStreetMap")
m1

In [ ]:
# Mapa con imagen satelital
m2 = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=14, height="500px"
)
m2.add_basemap("Esri.WorldImagery")
m2.add_title("San Juan de Pasto - Vista Satelital")
m2

## 5. Mapa con Puntos de Interes

In [ ]:
# Colores por tipo de lugar
colores_tipo = {
    "Plaza": "red",
    "Educacion": "blue",
    "Volcan": "orange",
    "Laguna": "cyan",
    "Aeropuerto": "purple",
    "Deportivo": "green",
    "Transporte": "yellow",
    "Comercial": "pink",
}

# Crear mapa con marcadores
m3 = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=12, height="600px"
)

m3.add_basemap("CartoDB.Positron")
m3.add_basemap("Esri.WorldImagery")

# Agregar marcadores
for nombre, info in LUGARES.items():
    color = colores_tipo.get(info["tipo"], "gray")
    m3.add_marker(
        location=[info["lat"], info["lon"]],
        popup=f"<b>{nombre}</b><br>{info['descripcion']}",
        icon_color=color,
    )

m3.add_layer_control()
m3

## 6. Mapa con Datos Vectoriales (GeoDataFrame)

In [ ]:
# Crear areas de ejemplo (barrios/zonas)
zonas = {
    "Centro Historico": box(-77.285, 1.210, -77.275, 1.220),
    "Zona Universitaria": box(-77.282, 1.215, -77.272, 1.225),
    "Zona Industrial": box(-77.278, 1.195, -77.260, 1.210),
    "Zona Residencial Norte": box(-77.295, 1.220, -77.275, 1.235),
}

gdf_zonas = gpd.GeoDataFrame(
    {
        "zona": list(zonas.keys()),
        "area_km2": [g.area * 111 * 111 for g in zonas.values()],  # Aproximacion
    },
    geometry=list(zonas.values()),
    crs="EPSG:4326",
)

print("Zonas de Pasto:")
gdf_zonas

In [ ]:
# Visualizar zonas
m4 = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=13, height="600px"
)

m4.add_basemap("OpenStreetMap")

# Agregar zonas con colores
m4.add_gdf(
    gdf_zonas,
    layer_name="Zonas de Pasto",
    style={"color": "blue", "fillOpacity": 0.3, "weight": 2},
)

# Agregar puntos de interes
m4.add_gdf(
    gdf_lugares,
    layer_name="Puntos de Interes",
    style={"color": "red", "fillColor": "yellow", "fillOpacity": 0.8},
)

m4.add_layer_control()
m4

## 7. Mapa Dividido (Split Map)

In [ ]:
# Mapa dividido para comparar capas base
m5 = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=14, height="600px"
)

m5.split_map(left_layer="OpenStreetMap", right_layer="Esri.WorldImagery")

m5

## 8. Mapa de Calor (Heatmap)

In [ ]:
# Crear datos aleatorios de ejemplo (podrian ser datos reales de poblacion, etc.)
np.random.seed(42)

# Generar puntos aleatorios alrededor del centro de Pasto
n_points = 200
lats = np.random.normal(PASTO_CENTER["lat"], 0.02, n_points)
lons = np.random.normal(PASTO_CENTER["lon"], 0.02, n_points)
valores = np.random.randint(1, 10, n_points)

# Crear datos para heatmap
heat_data = [[lat, lon, val] for lat, lon, val in zip(lats, lons, valores)]

# Crear mapa de calor
m6 = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=13, height="600px"
)

m6.add_basemap("CartoDB.DarkMatter")
m6.add_heatmap(heat_data, name="Densidad de Actividad", radius=15)

m6.add_layer_control()
m6

## 9. Mapa con Lineas (Rutas)

In [ ]:
# Crear ruta de ejemplo (recorrido turistico)
ruta_turistica = [
    [LUGARES["Plaza de Narino"]["lat"], LUGARES["Plaza de Narino"]["lon"]],
    [LUGARES["Universidad de Narino"]["lat"], LUGARES["Universidad de Narino"]["lon"]],
    [LUGARES["Estadio Libertad"]["lat"], LUGARES["Estadio Libertad"]["lon"]],
    [
        LUGARES["Terminal de Transportes"]["lat"],
        LUGARES["Terminal de Transportes"]["lon"],
    ],
    [
        LUGARES["Centro Comercial Unicentro"]["lat"],
        LUGARES["Centro Comercial Unicentro"]["lon"],
    ],
]

# Crear mapa
m7 = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=14, height="600px"
)

m7.add_basemap("OpenStreetMap")

# Agregar ruta
import folium

folium.PolyLine(ruta_turistica, weight=5, color="blue", opacity=0.8).add_to(m7)

# Agregar marcadores de la ruta
for i, punto in enumerate(ruta_turistica):
    m7.add_marker(location=punto, popup=f"Parada {i+1}")

m7

## 10. Mapa 3D del Volcan Galeras

In [ ]:
# Mapa del Volcan Galeras
galeras_center = [1.2208, -77.3592]

m8 = leafmap.Map(center=galeras_center, zoom=13, height="600px")

m8.add_basemap("OpenTopoMap")
m8.add_basemap("Esri.WorldImagery")

# Agregar circulo de zona de peligro (ejemplo)
import folium

folium.Circle(
    location=galeras_center,
    radius=5000,  # 5 km
    color="red",
    fill=True,
    fillColor="red",
    fillOpacity=0.2,
    popup="Zona de alto riesgo volcanico (5 km)",
).add_to(m8)

folium.Circle(
    location=galeras_center,
    radius=10000,  # 10 km
    color="orange",
    fill=True,
    fillColor="orange",
    fillOpacity=0.1,
    popup="Zona de riesgo moderado (10 km)",
).add_to(m8)

m8.add_marker(
    location=galeras_center,
    popup="<b>Volcan Galeras</b><br>Altitud: 4,276 m<br>Estado: Activo",
)

m8.add_layer_control()
m8

## 11. Mapa de la Laguna de la Cocha

In [ ]:
# Mapa de la Laguna de la Cocha
cocha_center = [1.1167, -77.1500]

m9 = leafmap.Map(center=cocha_center, zoom=12, height="600px")

m9.add_basemap("Esri.WorldImagery")

m9.add_marker(
    location=cocha_center,
    popup="<b>Laguna de la Cocha</b><br>Segunda laguna natural mas grande de Colombia<br>Sitio Ramsar",
)

# Isla La Corota
corota = [1.1189, -77.1628]
m9.add_marker(location=corota, popup="<b>Isla La Corota</b><br>Santuario de Flora")

m9

## 12. Exportar Mapas

In [ ]:
# Guardar mapa como HTML
import os

os.makedirs("./data", exist_ok=True)

# Crear mapa final con todos los elementos
m_final = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=12, height="700px"
)

m_final.add_basemap("OpenStreetMap")
m_final.add_basemap("Esri.WorldImagery")

# Agregar zonas
m_final.add_gdf(
    gdf_zonas, layer_name="Zonas", style={"color": "blue", "fillOpacity": 0.2}
)

# Agregar puntos
m_final.add_gdf(gdf_lugares, layer_name="Lugares de Interes")

m_final.add_layer_control()

# Guardar
m_final.to_html("./data/mapa_pasto_interactivo.html")
print("Mapa guardado en: ./data/mapa_pasto_interactivo.html")

m_final

## 13. Resumen

En este notebook hemos aprendido a:

1. **Crear mapas interactivos** con diferentes capas base
2. **Agregar marcadores y popups** con informacion
3. **Visualizar datos vectoriales** (puntos, poligonos, lineas)
4. **Crear mapas divididos** para comparaciones
5. **Generar mapas de calor** para visualizar densidades
6. **Exportar mapas** como archivos HTML

### Visualizaciones creadas para Pasto:

- Mapa de puntos de interes turisticos
- Zonas urbanas de la ciudad
- Mapa del Volcan Galeras con zonas de riesgo
- Mapa de la Laguna de la Cocha
- Rutas turisticas

### Proximos pasos:

- Integrar datos raster (imagenes satelitales)
- Crear dashboards interactivos
- Publicar mapas en la web